<a href="https://colab.research.google.com/github/pejmanrasti/Formation_institut_optique/blob/main/2_CNN_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

## Loading Training and Validation Data

we use the dataset [CalTech-101](http://www.vision.caltech.edu/Image_Datasets/Caltech101/), which contains around 9000 labeled images belonging to 101 object categories. However, we will exclude 5 of the categories which have the most images. This is in order to keep the class distribution fairly balanced (around 50-100) and constrained to a smaller number of images, around 2935.

# **Importing necessary Libraries**

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from tensorflow.keras.models import Sequential,Model # Model type to be used
from tensorflow.keras.layers import Dense, Activation, Dropout# Make Fully connected (FC) layers
from tensorflow.keras.utils import to_categorical # NumPy related tools
from tensorflow.keras.callbacks import TensorBoard  #Visulization of Accuracy and loss
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
! pip install livelossplot
from livelossplot import PlotLossesKeras

In [ ]:
from google.colab import drive
root = '/content/gdrive/'
drive.mount( root )

In [ ]:
!unzip gdrive/My\ Drive/data_Iran/101_ObjectCategories.zip

In [ ]:
DATADIR = "101_ObjectCategories"
CATEGORIES = os.listdir(DATADIR)
print(CATEGORIES)

In [ ]:
CATEGORIES = ['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']
print(CATEGORIES)

In [ ]:
training_data = []
IMG_SIZE_H=224 # you need to set up a numerical value here. Useful to resize to normalize data size
IMG_SIZE_W=224 # you need to set up a numerical value here. Useful to resize to normalize data size
def create_training_data():
    for category in CATEGORIES:  # do plants and weeds

        path = os.path.join(DATADIR,category)  # create path to the labels
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=plants 1=weeds

        for img in os.listdir(path):  # iterate over each image per plants and weeds
          if img.endswith('.jpg'):
            img_array = cv2.imread(os.path.join(path,img))  # convert to array
            new_array = cv2.resize(img_array, (IMG_SIZE_H, IMG_SIZE_W))  # resize to normalize data size
            training_data.append([new_array, class_num])  # add this to our training_data


In [ ]:
create_training_data()  # Calling the function for reading images and labels
print(len(training_data)) # Printing the size of the database

Preparation of data for importing to Keras

In [ ]:
random.shuffle(training_data)
X = []  # An Array for images
y = []  # An Array for labels

for features,label in training_data:   # Seperation of iamegs and labels
    X.append(features)
    y.append(label)
print(np.array(X).shape) # Print the size of the database

In [ ]:
X = np.array(X).reshape(-1, IMG_SIZE_H, IMG_SIZE_W, 3)  # Reshape data in a form that is suitable for keras
print(X.shape) # Print the size of the database

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5)) # Adjust figsize for better layout
for i, ax in enumerate([0, 10, 1000]):
    axes[i].imshow(X[ax,:,:,:])
    axes[i].set_title(f"Image {ax+1} - Class: {CATEGORIES[y[ax]]}") # Display class name
    axes[i].axis("off")

plt.tight_layout() # Adjust layout to prevent overlapping
plt.show()

### First training a neural net from scratch

Before doing the transfer learning, let's first build a neural network from scratch for doing classification on our dataset. This will give us a baseline to compare to our transfer-learned network later.

The network we will construct contains 4 alternating convolutional and max-pooling layers. After the last pooling layer, we will attach a fully-connected layer with 256 neurons then finally a softmax classification layer for our classes.



In [ ]:
# normalize data
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
# one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
# build the network
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

In [ ]:
plotlosses = PlotLossesKeras()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=50, batch_size=32,
          verbose=1,
          callbacks=[plotlosses])

In [ ]:
# build the network
model_e = Sequential() # Changed model to model_e

model_e.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:]))
model_e.add(Activation('relu'))
model_e.add(MaxPooling2D(pool_size=(2, 2)))

model_e.add(Conv2D(32, (3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling2D(pool_size=(2, 2)))

model_e.add(Conv2D(32, (3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling2D(pool_size=(2, 2)))

model_e.add(Conv2D(32, (3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling2D(pool_size=(2, 2)))

model_e.add(Flatten())
model_e.add(Dense(256))
model_e.add(Activation('relu'))

model_e.add(Dense(5))
model_e.add(Activation('softmax'))


In [ ]:
plotlosses_e = PlotLossesKeras()
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, verbose=1, mode='auto') #Stop training when a monitored metric has stopped improving.

checkpoint_filepath = 'checkpointCNN.keras'
Model_check = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto') #Callback to save the Keras model or model weights at some frequency.

In [ ]:
model_e.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_e.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=500, batch_size=32,
          verbose=1,
          callbacks=[plotlosses_e,early_stop, Model_check])

**Evaluation and Prediction**

We can use our model to make a prediction on new images.

In [ ]:
model_New_load = tf.keras.models.load_model('checkpointCNN.keras')
loss, accuracy = model_New_load.evaluate(X_test, y_test, verbose=0) #Evaluation of the model on the test dataset
print('Test loss:', loss)
print('Test accuracy:', accuracy)